In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init
from torch.utils.data import DataLoader
import torch.optim as optim

In [2]:
device='cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device=='cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
learning_rate=0.001
training_epochs=30
batch_size=100

In [4]:
mnist_train=dsets.MNIST(root='data/MNIST_data',
                        train=True,
                        transform=transforms.ToTensor(),
                        download=True)
mnist_test=dsets.MNIST(root='data/MNIST_data',
                        train=False,
                        transform=transforms.ToTensor(),
                        download=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 179105291.19it/s]


Extracting data/MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST_data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 129695603.67it/s]


Extracting data/MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST_data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 68500622.98it/s]


Extracting data/MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST_data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 22333562.45it/s]


Extracting data/MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST_data/MNIST/raw



In [5]:
data_loader=DataLoader(dataset=mnist_train,
                       batch_size=batch_size,
                       shuffle=True,
                       drop_last=True)

In [6]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1=nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer2=nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc=nn.Linear(7*7*64, 10, bias=True) # 완전연결층
        nn.init.xavier_uniform_(self.fc.weight) # 완전연결층 한정 가중치 초기화

    def forward(self, x):
        out=self.layer1(x)
        out=self.layer2(out)
        out=out.view(out.size(0),-1)
        out=self.fc(out)
        return out

In [7]:
model=CNN().to(device)
criterion=nn.CrossEntropyLoss().to(device)
optimizer=optim.Adam(model.parameters(), lr=learning_rate)

In [8]:
total_batch=len(data_loader)
print(total_batch)

600


In [9]:
for epoch in range(training_epochs):
    avg_cost=0
    avg_accuracy=0

    for X, Y in data_loader:
        X=X.to(device)
        Y=Y.to(device)

        optimizer.zero_grad()
        y_hat=model(X)
        cost=criterion(y_hat, Y)
        cost.backward()
        optimizer.step()

        avg_cost+=cost/total_batch
    print(epoch, avg_cost)

0 tensor(0.2257, device='cuda:0', grad_fn=<AddBackward0>)
1 tensor(0.0630, device='cuda:0', grad_fn=<AddBackward0>)
2 tensor(0.0462, device='cuda:0', grad_fn=<AddBackward0>)
3 tensor(0.0374, device='cuda:0', grad_fn=<AddBackward0>)
4 tensor(0.0314, device='cuda:0', grad_fn=<AddBackward0>)
5 tensor(0.0262, device='cuda:0', grad_fn=<AddBackward0>)
6 tensor(0.0217, device='cuda:0', grad_fn=<AddBackward0>)
7 tensor(0.0185, device='cuda:0', grad_fn=<AddBackward0>)
8 tensor(0.0164, device='cuda:0', grad_fn=<AddBackward0>)
9 tensor(0.0135, device='cuda:0', grad_fn=<AddBackward0>)
10 tensor(0.0100, device='cuda:0', grad_fn=<AddBackward0>)
11 tensor(0.0099, device='cuda:0', grad_fn=<AddBackward0>)
12 tensor(0.0087, device='cuda:0', grad_fn=<AddBackward0>)
13 tensor(0.0063, device='cuda:0', grad_fn=<AddBackward0>)
14 tensor(0.0076, device='cuda:0', grad_fn=<AddBackward0>)
15 tensor(0.0069, device='cuda:0', grad_fn=<AddBackward0>)
16 tensor(0.0059, device='cuda:0', grad_fn=<AddBackward0>)
17 tens

In [10]:
with torch.no_grad():
  X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
  Y_test = mnist_test.test_labels.to(device)

  prediction = model(X_test)
  correct_prediction = torch.argmax(prediction, 1) == Y_test
  accuracy = correct_prediction.float().mean()
  print(accuracy.item())

0.9839999675750732


/usr/local/lib/python3.10/dist-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.10/dist-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
